- use f1 instead
- add more data and see

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.4MB 22.2MB/s 
     |████████████████████████████████| 890kB 43.2MB/s 
     |████████████████████████████████| 2.9MB 45.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=65ffae3af98619bec8c50f2bec760db96d93380ae639d997a3ef6dcf9988255f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
import re
import sys
warnings.filterwarnings('ignore')

## Import the dataset

---



In [ ]:
url = 'https://raw.githubusercontent.com/CS410Fall2020/ClassificationCompetition/main/data/train.jsonl'
df = pd.read_json(url, lines=True)

url_test = 'https://raw.githubusercontent.com/CS410Fall2020/ClassificationCompetition/main/data/test.jsonl'
df_test = pd.read_json(url_test, lines=True)


# We dont use the context
del df["context"]
del df_test["context"]

# TEST
df_test_id = df_test["id"]
df_test = df_test["response"]
print(df_test)

# Place the label column at the end
response = df['response']
df.drop(labels=['response'], axis=1,inplace = True)
df.insert(0, 'response', response)

# Labels to numbers
print(df[0:3])
print(df[3000:3003])
df['label'] = pd.factorize(df['label'])[0]
print(df[0:3])
print(df[3000:3003])

0       @USER @USER @USER My 3 year old , that just fi...
1       @USER @USER How many verifiable lies has he to...
2       @USER @USER @USER Maybe Docs just a scrub of a...
3       @USER @USER is just a cover up for the real ha...
4       @USER @USER @USER The irony being that he even...
                              ...                        
1795    @USER @USER @USER is definitely the best out t...
1796    @USER @USER Ye let her out run wild and infect...
1797    @USER @USER @USER Thanks for that , I would ha...
1798    @USER @USER @USER Yes also #found this on #new...
1799    @USER @USER @USER you still need to send the l...
Name: response, Length: 1800, dtype: object
                                            response    label
0  @USER @USER @USER I don't get this .. obviousl...  SARCASM
1  @USER @USER trying to protest about . Talking ...  SARCASM
2  @USER @USER @USER He makes an insane about of ...  SARCASM
                                               response        label
3

## Load pretrain DistilBERT

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

## Prepare the dataset

1. Delete @USER from tweets
2. remove # from Hashtags?
3. we keep quotes
4. Delete URL ?????????????????????????????????????????????????????????????
5. Use BERT tokenizer (which turns it to uncased)




In [ ]:
# GhOSH dataset
#0    26736
#1    24453
url_ghosh = 'https://raw.githubusercontent.com/MirunaPislar/Sarcasm-Detection/master/res/datasets/ghosh/train.txt'
df_ghosh = pd.read_csv(url_ghosh, delimiter='\t', names=['trainsen','labels','tweets'])

df_ghosh = df_ghosh[0:4000]

del df_ghosh['trainsen']

df_ghosh['tweets'] = df_ghosh['tweets'].apply((lambda x: x.replace('#sarcasm', '')))
df_ghosh['tweets'] = df_ghosh['tweets'].apply((lambda x: x.replace('#', '')))
df_ghosh['tweets'] = df_ghosh['tweets'].apply((lambda x: x.replace('@', '')))
df_ghosh['tweets'] = df_ghosh['tweets'].apply((lambda x: re.sub("https*\S+", " ", x)))
df_ghosh['tweets'] = df_ghosh['tweets'].apply((lambda x: x.replace('  ', ' ')))
df_ghosh['tweets'] = df_ghosh['tweets'].apply((lambda x: x.replace('  ', ' ')))
df_ghosh['tweets'] = df_ghosh['tweets'].apply((lambda x: x.replace('  ', ' ')))

#flip labels 1 to 0 and 0 to 1
df_ghosh['labels'] = df_ghosh['labels'].apply((lambda x: 1 - x))

tokenized_ghosh = df_ghosh['tweets'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
#TRAIN
df['response'] = df['response'].apply((lambda x: x.replace('@USER', '')))
df['response'] = df['response'].apply((lambda x: x.replace('<URL>', '')))
df['response'] = df['response'].apply((lambda x: x.strip()))
df['response'] = df['response'].apply((lambda x: x.replace('  ', ' ')))
df['response'] = df['response'].apply((lambda x: x.replace('  ', ' ')))
df['response'] = df['response'].apply((lambda x: x.replace('  ', ' ')))
df['response'] = df['response'].apply((lambda x: x.replace('#', '')))

batch_1 = df
print(batch_1[0:3])
print(batch_1[3000:3003])

tokenized = batch_1['response'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

#TEST

df_test = df_test.apply((lambda x: x.replace('@USER', '')))
df_test = df_test.apply((lambda x: x.replace('<URL>', '')))
df_test = df_test.apply((lambda x: x.strip()))
df_test = df_test.apply((lambda x: x.replace('  ', ' ')))
df_test = df_test.apply((lambda x: x.replace('  ', ' ')))
df_test = df_test.apply((lambda x: x.replace('  ', ' ')))
df_test = df_test.apply((lambda x: x.replace('#', '')))

tokenized_test = df_test.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))





                                            response  label
0  I don't get this .. obviously you do care or y...      0
1  trying to protest about . Talking about him an...      0
2  He makes an insane about of money from the MOV...      0
                                               response  label
3000  or you could look at it from a positive point ...      1
3001  I ’ ve actually washed the lot . It ’ s taken ...      1
3002  scared too . Imagining if I were an immigrant ...      1


### Padding
Fill with zeros so that all sentences have the same size

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

for i in tokenized_test.values:
    if len(i) > max_len:
        max_len = len(i)

for i in tokenized_ghosh.values:
    if len(i) > max_len:
        max_len = len(i)

        

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
np.array(padded).shape

padded_ghosh = np.array([i + [0]*(max_len-len(i)) for i in tokenized_ghosh.values])
np.array(padded).shape

#TEST
padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])
np.array(padded_test).shape

(1800, 87)

### Masking
Tell BERT to ignore the padded zeros

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(5000, 87)

## Run the model

### Run BERT
get the 5000 tweets already through BERT

In [ ]:
input_ids = torch.tensor(padded[0:1000])  
attention_mask = np.where(padded[0:1000] != 0, 1, 0)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states1 = model(input_ids, attention_mask=attention_mask)

In [ ]:
input_ids = torch.tensor(padded[1000:2000])  
attention_mask = np.where(padded[1000:2000] != 0, 1, 0)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states2 = model(input_ids, attention_mask=attention_mask)

In [ ]:
input_ids = torch.tensor(padded[2000:3000]) 
attention_mask = np.where(padded[2000:3000] != 0, 1, 0) 
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states3 = model(input_ids, attention_mask=attention_mask)

In [ ]:
input_ids = torch.tensor(padded[3000:4000])  
attention_mask = np.where(padded[3000:4000] != 0, 1, 0)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states4 = model(input_ids, attention_mask=attention_mask)

In [ ]:
input_ids = torch.tensor(padded[4000:5000])  
attention_mask = np.where(padded[4000:5000] != 0, 1, 0)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states5 = model(input_ids, attention_mask=attention_mask)

In [ ]:
input_ids = torch.tensor(padded_ghosh[0:2000])  
attention_mask = np.where(padded_ghosh[0:2000] != 0, 1, 0)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    ghosh_hidden_states1 = model(input_ids, attention_mask=attention_mask)

In [ ]:
input_ids = torch.tensor(padded_ghosh[2000:4000])  
attention_mask = np.where(padded_ghosh[2000:4000] != 0, 1, 0)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    ghosh_hidden_states2 = model(input_ids, attention_mask=attention_mask)

In [ ]:
#TEST
input_ids = torch.tensor(padded_test)  
attention_mask = np.where(padded_test != 0, 1, 0)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states_test = model(input_ids, attention_mask=attention_mask)

In [ ]:
frames = [last_hidden_states1, last_hidden_states2, last_hidden_states3, last_hidden_states4, last_hidden_states5, ghosh_hidden_states1, ghosh_hidden_states2]
#last_hidden_states = pd.concat(frames)
print(last_hidden_states1)

BaseModelOutput(last_hidden_state=tensor([[[ 8.6516e-02, -1.5730e-02,  3.1538e-02,  ..., -4.7312e-02,
           3.4480e-01,  2.2681e-01],
         [ 5.0332e-01,  2.5628e-01,  1.9403e-02,  ...,  3.0671e-01,
           4.2098e-01,  2.9953e-01],
         [ 9.0519e-02,  5.8046e-01,  6.1210e-01,  ...,  2.3946e-01,
           1.7250e-01, -2.1856e-01],
         ...,
         [ 3.4564e-01,  1.6241e-01,  3.5609e-01,  ..., -2.2443e-01,
          -9.4811e-02, -1.1109e-01],
         [ 2.7448e-01, -1.0026e-01,  9.1260e-02,  ..., -9.9428e-02,
          -5.6052e-02, -1.8616e-01],
         [ 2.8177e-01, -1.0023e-01,  9.7833e-02,  ..., -3.0372e-02,
          -5.0148e-02, -1.4436e-01]],

        [[ 1.2924e-01,  1.8292e-01, -6.3054e-02,  ..., -1.5129e-01,
           4.9060e-01,  1.0455e-01],
         [ 2.4602e-01,  1.8424e-01, -1.4749e-01,  ...,  2.7233e-01,
           3.4833e-01, -1.5184e-01],
         [ 3.7892e-01,  3.0204e-01, -2.1360e-01,  ..., -4.7395e-05,
           2.7753e-01, -4.7282e-02],
     

In [ ]:
features1 = last_hidden_states1[0][:,0,:].numpy()
features2 = last_hidden_states2[0][:,0,:].numpy()
features3 = last_hidden_states3[0][:,0,:].numpy()
features4 = last_hidden_states4[0][:,0,:].numpy()
features5 = last_hidden_states5[0][:,0,:].numpy()
features6 = ghosh_hidden_states1[0][:,0,:].numpy()
features7 = ghosh_hidden_states2[0][:,0,:].numpy()

aux_features = np.concatenate((features1, features2), axis=0)
print(aux_features.shape)

aux_features = np.concatenate((aux_features, features3), axis=0)
aux_features = np.concatenate((aux_features, features4), axis=0)
aux_features = np.concatenate((aux_features, features5), axis=0)
aux_features = np.concatenate((aux_features, features6), axis=0)
aux_features = np.concatenate((aux_features, features7), axis=0)
print(aux_features.shape)

features = aux_features

labels = batch_1['label']
ghosh_labels = df_ghosh['labels']
labels = np.concatenate((labels, ghosh_labels), axis=0)
print("hola")
print(labels.shape)

#TEST
features_test = last_hidden_states_test[0][:,0,:].numpy()
print(features_test.shape)

(2000, 768)
(9000, 768)
hola
(9000,)
(1800, 768)


### Split train/test data

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
train_labels.shape

(6750,)

### Logistic Regression and others

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    y_true = y_true.astype(np.float32)
    y_pred = y_pred.astype(np.float32)
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# 1
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
lr_clf.score(test_features, test_labels)
prediction = lr_clf.predict(test_features)

print(f1_m(true_labels, prediction))

tf.Tensor(0.71374846, shape=(), dtype=float32)


In [ ]:
# 2
from sklearn.svm import LinearSVC
lsvc = LinearSVC()
lsvc.fit(train_features, train_labels)
lsvc.score(test_features, test_labels)

0.6871111111111111

In [ ]:
# 4
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(train_features, train_labels)
gnb.score(test_features, test_labels)

0.6497777777777778

In [ ]:
# 3
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 250)
rfc.fit(train_features, train_labels)
rfc.score(test_features, test_labels)

0.6897777777777778

In [ ]:
from tensorflow import keras
import tensorflow as tf

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(768,)),
    keras.layers.Dense(64, activation=tf.nn.relu),
	keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


model.fit(train_features, train_labels, epochs=20, batch_size=1)
test_loss, test_acc = model.evaluate(test_features, test_labels)
print('Test accuracy:', test_acc)


Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_15 (Flatten)         (None, 768)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 64)                49216     
_________________________________________________________________
dense_46 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_47 (Dense)             (None, 1)                 65        
Total params: 53,441
Trainable params: 53,441
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
6750/6750 [==============================] - 12s 2ms/step - loss: 0.6174 - accuracy: 0.6437
Epoch 2/20
6750/6750 [==============================] - 12s 2ms/step - loss: 0.5889 - accuracy: 0.6763
Epoch 3/20
6750/6750 [===================

KeyboardInterrupt: ignored

In [ ]:
# CROS VALIDATION
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import KFold

kfold = KFold(n_splits = 5)

acc_per_fold = []
loss_per_fold = []
fold_no = 1
for train, test in kfold.split(features, labels):
  model = keras.Sequential([
    keras.layers.Flatten(input_shape=(768,)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
  ])

  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  print(f'Training for fold {fold_no} ...')

  history = model.fit(features[train], labels[train],
              batch_size=1,
              epochs=2, 
              verbose=1)
  
  scores = model.evaluate(features[test], labels[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  pred = model.predict_classes(features[test])
  pred = np.transpose(pred)
  print(labels[test])
  print(pred[0])
  print(f1_m(labels[test], pred[0]))

  # Increase fold number
  fold_no = fold_no + 1

print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')


Training for fold 1 ...
Epoch 1/2
7200/7200 [==============================] - 13s 2ms/step - loss: 0.5892 - accuracy: 0.6756
Epoch 2/2
7200/7200 [==============================] - 14s 2ms/step - loss: 0.5608 - accuracy: 0.6929
Score for fold 1: loss of 0.8746541142463684; accuracy of 22.88888841867447%
[0 0 0 ... 0 0 0]
[1 1 1 ... 1 0 1]
tf.Tensor(0.0, shape=(), dtype=float32)
Training for fold 2 ...
Epoch 1/2
5095/7200 [====================>.........] - ETA: 3s - loss: 0.6504 - accuracy: 0.6063

KeyboardInterrupt: ignored

[64,64,1]
Accuracy: 71.75999939441681 (+- 9.03495327163218)
Loss: 1.0835366070270538

## Evaluate the model

In [ ]:
lr_clf.score(test_features, test_labels)

0.6955555555555556

In [ ]:
lr_clf_test = LogisticRegression()
lr_clf_test.fit(features, labels)

lr_clf_test.score(features, labels)

0.7454444444444445

In [ ]:
lsvc_test = LinearSVC()
lsvc_test.fit(features, labels)

lsvc_test.score(features, labels)

0.7588888888888888

In [ ]:
gnb_test = GaussianNB()
gnb_test.fit(features, labels)

gnb_test.score(features, labels)

NameError: ignored

In [ ]:
rfc_test = RandomForestClassifier(n_estimators = 250)
rfc_test.fit(features, labels)

rfc_test.score(features, labels)

0.9998888888888889

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(768,)),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
  ])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(features, labels, epochs=30, batch_size=1)
test_loss, test_acc = model.evaluate(test_features, test_labels)
print('Test accuracy:', test_acc)

Epoch 1/30
9000/9000 [==============================] - 15s 2ms/step - loss: 0.6164 - accuracy: 0.6499
Epoch 2/30
9000/9000 [==============================] - 16s 2ms/step - loss: 0.5842 - accuracy: 0.6824
Epoch 3/30
9000/9000 [==============================] - 15s 2ms/step - loss: 0.5690 - accuracy: 0.6968
Epoch 4/30
9000/9000 [==============================] - 16s 2ms/step - loss: 0.5603 - accuracy: 0.6996
Epoch 5/30
9000/9000 [==============================] - 16s 2ms/step - loss: 0.5528 - accuracy: 0.7021
Epoch 6/30
9000/9000 [==============================] - 16s 2ms/step - loss: 0.5422 - accuracy: 0.7106
Epoch 7/30
9000/9000 [==============================] - 16s 2ms/step - loss: 0.5346 - accuracy: 0.7187
Epoch 8/30
9000/9000 [==============================] - 16s 2ms/step - loss: 0.5265 - accuracy: 0.7231
Epoch 9/30
9000/9000 [==============================] - 16s 2ms/step - loss: 0.5201 - accuracy: 0.7298
Epoch 10/30
9000/9000 [==============================] - 15s 2ms/step - l

In [ ]:
import sys

np.set_printoptions(threshold=sys.maxsize)
#np.set_printoptions(threshold=1000)
prediction = model.predict_classes(features_test)
print(prediction)
print(prediction.shape)

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]


In [ ]:
from google.colab import files

with open('Example2.txt', 'w') as writefile:
    n = 0
    sarcasm = 0
    not_sarcasm = 0
    for x in prediction:
      n = n + 1
      if x == 1:
        writefile.write("twitter_{},NOT_SARCASM\n".format(n))
        sarcasm += 1
      else:
        writefile.write("twitter_{},SARCASM\n".format(n))
        not_sarcasm +=1

    print(sarcasm)
    print(not_sarcasm)  


870
930


In [ ]:
files.download('Example2.txt') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
asd = 1
print("asdfs",asd,"asdf")
print("If there was a birth every 7 seconds, there would be:{}births".format(asd))

asdfs 1 asdf
If there was a birth every 7 seconds, there would be:1births
